# (Optional) Colab Setup
If you aren't using Colab, you can delete the following code cell. This is just to help students with mounting to Google Drive to access the other .py files and downloading the data, which is a little trickier on Colab than on your local machine using Jupyter. 

In [115]:
# you will be prompted with a window asking to grant permissions
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [116]:
# fill in the path in your Google Drive in the string below. Note: do not escape slashes or spaces
import os
datadir = "/content/drive/My Drive/Spring2023/cs444/assignment1/"
if not os.path.exists(datadir):
  !ln -s "/content/drive/My Drive/Spring2023/cs444/assignment1/" $datadir
os.chdir(datadir)
!pwd

/content/drive/My Drive/Spring2023/cs444/assignment1


In [117]:
# downloading Fashion-MNIST
import os
os.chdir(os.path.join(datadir,"fashion-mnist/"))
!chmod +x ./get_data.sh
!./get_data.sh
os.chdir(datadir)

--2023-02-15 05:33:11--  https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/t10k-images-idx3-ubyte.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4422102 (4.2M) [application/octet-stream]
Saving to: ‘t10k-images-idx3-ubyte.gz.22’

t10k-images-idx3-ub 100%[===================>]   4.22M  --.-KB/s    in 0.1s    

2023-02-15 05:33:11 (42.3 MB/s) - ‘t10k-images-idx3-ubyte.gz.22’ saved [4422102/4422102]

--2023-02-15 05:33:11--  https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/t10k-labels-idx1-ubyte.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.

# Imports

In [118]:
import random
import numpy as np
from data_process import get_FASHION_data, get_RICE_data
from scipy.spatial import distance
from models import Perceptron, SVM, Softmax, Logistic
from kaggle_submission import output_submission_csv
%matplotlib inline

# For auto-reloading external modules
# See http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading Fashion-MNIST

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 60000]
, TEST_IMAGES = 10000

In [119]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 50000
VAL_IMAGES = 10000
normalize = True

In [120]:
data = get_FASHION_data(TRAIN_IMAGES, VAL_IMAGES, normalize=normalize)
X_train_fashion, y_train_fashion = data['X_train'], data['y_train']
X_val_fashion, y_val_fashion = data['X_val'], data['y_val']
X_test_fashion, y_test_fashion = data['X_test'], data['y_test']
n_class_fashion = len(np.unique(y_test_fashion))

# Loading Rice

In [121]:
# loads train / test / val splits of 80%, 20%, 20% 
data = get_RICE_data()
X_train_RICE, y_train_RICE = data['X_train'], data['y_train']
X_val_RICE, y_val_RICE = data['X_val'], data['y_val']
X_test_RICE, y_test_RICE = data['X_test'], data['y_test']
n_class_RICE = len(np.unique(y_test_RICE))

print("Number of train samples: ", X_train_RICE.shape[0])
print("Number of val samples: ", X_val_RICE.shape[0])
print("Number of test samples: ", X_test_RICE.shape[0])

Number of train samples:  10911
Number of val samples:  3637
Number of test samples:  3637


### Get Accuracy

This function computes how well your model performs using accuracy as a metric.

In [122]:
def get_acc(pred, y_test):
    return np.sum(y_test == pred) / len(y_test) * 100

# Perceptron

Perceptron has 2 hyperparameters that you can experiment with:
- **Learning rate** - controls how much we change the current weights of the classifier during each update. We set it at a default value of 0.5, but you should experiment with different values. We recommend changing the learning rate by factors of 10 and observing how the performance of the classifier changes. You should also try adding a **decay** which slowly reduces the learning rate over each epoch.
- **Number of Epochs** - An epoch is a complete iterative pass over all of the data in the dataset. During an epoch we predict a label using the classifier and then update the weights of the classifier according to the perceptron update rule for each sample in the training set. You should try different values for the number of training epochs and report your results.

You will implement the Perceptron classifier in the **models/perceptron.py**

The following code: 
- Creates an instance of the Perceptron classifier class 
- The train function of the Perceptron class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy


In [ ]:
"""Perceptron model."""

import numpy as np


class Perceptron:
  def __init__(self, n_class: int, lr: float, epochs: int):
    """Initialize a new classifier.

    Parameters:
        n_class: the number of classes
        lr: the learning rate
        epochs: the number of epochs to train for
    """
    self.w = None
    self.lr = lr
    self.epochs = epochs
    self.n_class = n_class

  def train(self, X_train: np.ndarray, y_train: np.ndarray):
    """Train the classifier.

    Use the perceptron update rule as introduced in the Lecture.

    Parameters:
        X_train: a number array of shape (N, D) containing training data;
            N examples with D dimensions
        y_train: a numpy array of shape (N,) containing training labels
    """
    N, D = X_train.shape
    C = self.n_class
    self.w = np.random.randn(C, D)

    for epoch in range(self.epochs):
      for i in range(N):
        w_yi = self.w[y_train[i]] @ X_train[i]
        for c in range(C):
          w_c = self.w[c] @ X_train[i]
          if w_c > w_yi:
            self.w[y_train[i]] += (self.lr * X_train[i])
            self.w[c] -= (self.lr * X_train[i])

        self.lr *= np.exp(-1 * (epoch * 0.001))

  def predict(self, X_test: np.ndarray) -> np.ndarray:
    """Use the trained weights to predict labels for test data points.

    Parameters:
        X_test: a numpy array of shape (N, D) containing testing data;
            N examples with D dimensions

    Returns:
        predicted labels for the data in X_test; a 1-dimensional array of
            length N, where each element is an integer giving the predicted
            class.
    """
    # N, D = X_test.shape
    # self.w = self.w.reshape(D, 1)
    pred = np.dot(X_test, self.w.T)
    return np.argmax(pred, axis=1)


## Train Perceptron on Fashion-MNIST

In [ ]:
lr = 0.5
n_epochs = 10

percept_fashion = Perceptron(n_class_fashion, lr, n_epochs)
percept_fashion.train(X_train_fashion, y_train_fashion)

In [ ]:
pred_percept = percept_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_fashion)))

The training accuracy is given by: 84.234000


### Validate Perceptron on Fashion-MNIST

In [ ]:
pred_percept = percept_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_fashion)))

The validation accuracy is given by: 83.110000


### Test Perceptron on Fashion-MNIST

In [ ]:
pred_percept = percept_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_fashion)))

The testing accuracy is given by: 81.910000


### Perceptron_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy, output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [ ]:
output_submission_csv('kaggle/perceptron_submission_fashion.csv', percept_fashion.predict(X_test_fashion))

## Train Perceptron on Rice

In [ ]:
lr = 0.5
n_epochs = 10

percept_RICE = Perceptron(n_class_RICE, lr, n_epochs)
percept_RICE.train(X_train_RICE, y_train_RICE)

In [ ]:
pred_percept = percept_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_RICE)))

The training accuracy is given by: 99.853359


### Validate Perceptron on Rice

In [ ]:
pred_percept = percept_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_RICE)))

The validation accuracy is given by: 99.807534


### Test Perceptron on Rice

In [ ]:
pred_percept = percept_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_RICE)))

The testing accuracy is given by: 99.725048


# Support Vector Machines (with SGD)

Next, you will implement a "soft margin" SVM. In this formulation you will maximize the margin between positive and negative training examples and penalize margin violations using a hinge loss.

We will optimize the SVM loss using SGD. This means you must compute the loss function with respect to model weights. You will use this gradient to update the model weights.

SVM optimized with SGD has 3 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Epochs** - similar to as defined above in Perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case it is a coefficient on the term which maximizes the margin. You could try different values. The default value is set to 0.05.

You will implement the SVM using SGD in the **models/svm.py**

The following code: 
- Creates an instance of the SVM classifier class 
- The train function of the SVM class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

In [165]:
"""Support Vector Machine (SVM) model."""

import numpy as np


class SVM:
  def __init__(self, n_class: int, lr: float, epochs: int, reg_const: float):
    """Initialize a new classifier.

    Parameters:
        n_class: the number of classes
        lr: the learning rate
        epochs: the number of epochs to train for
        reg_const: the regularization constant
    """
    self.w = None  # TODO: change this
    self.lr = lr
    self.epochs = epochs
    self.reg_const = reg_const
    self.n_class = n_class
    self.batch_size = 256

  def calc_gradient(self, X_train: np.ndarray, y_train: np.ndarray) -> np.ndarray:
    """Calculate gradient of the svm hinge loss.

    Inputs have dimension D, there are C classes, and we operate on
    mini-batches of N examples.

    Parameters:
        X_train: a numpy array of shape (N, D) containing a mini-batch
            of data
        y_train: a numpy array of shape (N,) containing training labels;
            y[i] = c means that X[i] has label c, where 0 <= c < C

    Returns:
        the gradient with respect to weights w; an array of the same shape
            as w
    """
    # TODO: implement me
    N, D = X_train.shape
    C = self.n_class
    grad = np.zeros((C, D))

    for i in range(N):
      scores = self.w @ X_train[i]
      correct_class_score = scores[y_train[i]]
      for j in range(C):
        if j == y_train[i]:
          continue
        margin = scores[j] - correct_class_score + 1
        if margin > 0:
          grad[j, :] += X_train[i, :]
          grad[y_train[i], :] -= X_train[i, :]

    grad /= N
    grad += 2 * self.reg_const * self.w

    return grad

  def train(self, X_train: np.ndarray, y_train: np.ndarray):
    """Train the classifier.

    Hint: operate on mini-batches of data for SGD.

    Parameters:
        X_train: a numpy array of shape (N, D) containing training data;
            N examples with D dimensions
        y_train: a numpy array of shape (N,) containing training labels
    """
    # TODO: implement me
    N, D = X_train.shape
    C = self.n_class
    self.w = np.random.randn(C, D)
    num_batch = N // self.batch_size

    for epoch in range(self.epochs):
      for batch in range(num_batch):
        i = np.random.choice(N, self.batch_size, replace=False)
        x_i = X_train[i]
        y_i = y_train[i]
        grad = self.calc_gradient(x_i, y_i)
        self.w -= self.lr * grad
      self.lr *= np.exp(-1 * (epoch))

  def predict(self, X_test: np.ndarray) -> np.ndarray:
    """Use the trained weights to predict labels for test data points.

    Parameters:
        X_test: a numpy array of shape (N, D) containing testing data;
            N examples with D dimensions

    Returns:
        predicted labels for the data in X_test; a 1-dimensional array of
            length N, where each element is an integer giving the predicted
            class.
    """
    # TODO: implement me
  
    pred = np.dot(X_test, self.w.T)
    return np.argmax(pred, axis=1) # min


## Train SVM on Fashion-MNIST

In [166]:
lr = 0.05
n_epochs = 25
reg_const = 0.005
svm_fashion = SVM(n_class_fashion, lr, n_epochs, reg_const)
svm_fashion.train(X_train_fashion, y_train_fashion)

In [167]:
pred_svm = svm_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_fashion)))

The training accuracy is given by: 84.164000


### Validate SVM on Fashion-MNIST

In [168]:
pred_svm = svm_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_fashion)))

The validation accuracy is given by: 82.720000


### Test SVM on Fashion-MNIST

In [169]:
pred_svm = svm_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_fashion)))

The testing accuracy is given by: 82.080000


### SVM_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [133]:
output_submission_csv('kaggle/svm_submission_fashion.csv', svm_fashion.predict(X_test_fashion))

## Train SVM on Rice

In [23]:
lr = 0.001
n_epochs = 100
reg_const = 0.5

svm_RICE = SVM(n_class_RICE, lr, n_epochs, reg_const)
svm_RICE.train(X_train_RICE, y_train_RICE)

In [24]:
pred_svm = svm_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_RICE)))

The training accuracy is given by: 99.688388


### Validate SVM on Rice

In [25]:
pred_svm = svm_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_RICE)))

The validation accuracy is given by: 99.642563


## Test SVM on Rice

In [26]:
pred_svm = svm_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_RICE)))

The testing accuracy is given by: 99.560077


# Softmax Classifier (with SGD)

Next, you will train a Softmax classifier. This classifier consists of a linear function of the input data followed by a softmax function which outputs a vector of dimension C (number of classes) for each data point. Each entry of the softmax output vector corresponds to a confidence in one of the C classes, and like a probability distribution, the entries of the output vector sum to 1. We use a cross-entropy loss on this sotmax output to train the model. 

Check the following link as an additional resource on softmax classification: http://cs231n.github.io/linear-classify/#softmax

Once again we will train the classifier with SGD. This means you need to compute the gradients of the softmax cross-entropy loss function according to the weights and update the weights using this gradient. Check the following link to help with implementing the gradient updates: https://deepnotes.io/softmax-crossentropy

The softmax classifier has 3 hyperparameters that you can experiment with:
- **Learning rate** - As above, this controls how much the model weights are updated with respect to their gradient.
- **Number of Epochs** - As described for perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case, we minimize the L2 norm of the model weights as regularization, so the regularization constant is a coefficient on the L2 norm in the combined cross-entropy and regularization objective.

You will implement a softmax classifier using SGD in the **models/softmax.py**

The following code: 
- Creates an instance of the Softmax classifier class 
- The train function of the Softmax class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

In [171]:
"""Softmax model."""

import numpy as np


class Softmax:
  def __init__(self, n_class: int, lr: float, epochs: int, reg_const: float):
    """Initialize a new classifier.

    Parameters:
        n_class: the number of classes
        lr: the learning rate
        epochs: the number of epochs to train for
        reg_const: the regularization constant
    """
    self.w = None  # TODO: change this
    self.lr = lr
    self.epochs = epochs
    self.reg_const = reg_const
    self.n_class = n_class

  def calc_gradient(self, X_train: np.ndarray, y_train: np.ndarray) -> np.ndarray:
    """Calculate gradient of the softmax loss.

    Inputs have dimension D, there are C classes, and we operate on
    mini-batches of N examples.

    Parameters:
        X_train: a numpy array of shape (N, D) containing a mini-batch
            of data
        y_train: a numpy array of shape (N,) containing training labels;
            y[i] = c means that X[i] has label c, where 0 <= c < C

    Returns:
        gradient with respect to weights w; an array of same shape as w
    """
    # TODO: implement me
    gradient = self.reg_const * self.w
    const_k = np.max(self.score, axis = 1, keepdims=True)
    self.score = np.exp(self.score-const_k)
    self.score /= np.sum(self.score,axis=1, keepdims=True)
    
    
    ground_truth = self.score[np.arange(X_train.shape[0]), y_train]
    self.score[np.arange(X_train.shape[0]), y_train] = 0
    
    for i in range(X_train.shape[0]):
      gradient += X_train[i][:,np.newaxis] * self.score[i]
      gradient[:, y_train[i]] += (ground_truth[i] - 1) * X_train[i]
      
    return gradient

  def train(self, X_train: np.ndarray, y_train: np.ndarray):
    """Train the classifier.

    Hint: operate on mini-batches of data for SGD.

    Parameters:
        X_train: a numpy array of shape (N, D) containing training data;
            N examples with D dimensions
        y_train: a numpy array of shape (N,) containing training labels
    """
    # TODO: implement me
    self.w = np.random.rand(X_train.shape[1],10)
    betch_size = 128
    for e in range(self.epochs):
      self.lr /= (1+e)
      for i in range(X_train.shape[0]):
        x = X_train[i*betch_size: (i+1) * betch_size]
        y = y_train[i*betch_size: (i+1) * betch_size]
        self.score = x.dot(self.w)
        self.w -= self.lr * self.calc_gradient(x,y)
    return

  def predict(self, X_test: np.ndarray) -> np.ndarray:
    """Use the trained weights to predict labels for test data points.

    Parameters:
        X_test: a numpy array of shape (N, D) containing testing data;
            N examples with D dimensions

    Returns:
        predicted labels for the data in X_test; a 1-dimensional array of
            length N, where each element is an integer giving the predicted
            class.
    """
    # TODO: implement me
    pred = X_test.dot(self.w).argmax(axis=1)
    return pred


## Train Softmax on Fashion-MNIST

In [ ]:
lr = 0.5
n_epochs = 10
reg_const = 0.5

softmax_fashion = Softmax(n_class_fashion, lr, n_epochs, reg_const)
softmax_fashion.train(X_train_fashion, y_train_fashion)

In [ ]:
pred_softmax = softmax_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_fashion)))

The training accuracy is given by: 85.258000


### Validate Softmax on Fashion-MNIST

In [ ]:
pred_softmax = softmax_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_fashion)))

The validation accuracy is given by: 83.470000


### Testing Softmax on Fashion-MNIST

In [ ]:
pred_softmax = softmax_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_fashion)))

The testing accuracy is given by: 82.820000


### Softmax_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [ ]:
output_submission_csv('kaggle/softmax_submission_fashion.csv', softmax_fashion.predict(X_test_fashion))

## Train Softmax on Rice

In [172]:
lr = 0.01
n_epochs = 50
reg_const = 0.9

#0.01, 50, 0.9

softmax_RICE = Softmax(n_class_RICE, lr, n_epochs, reg_const)
softmax_RICE.train(X_train_RICE, y_train_RICE)

In [173]:
pred_softmax = softmax_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_RICE)))

The training accuracy is given by: 99.789204


### Validate Softmax on Rice

In [174]:
pred_softmax = softmax_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_RICE)))

The validation accuracy is given by: 99.835029


### Testing Softmax on Rice

In [175]:
pred_softmax = softmax_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_RICE)))

The testing accuracy is given by: 99.642563


# Logistic Classifier

The Logistic Classifier has 2 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Number of Epochs** - As described for perceptron.
- **Threshold** - The decision boundary of the classifier.


You will implement the Logistic Classifier in the **models/logistic.py**

The following code: 
- Creates an instance of the Logistic classifier class 
- The train function of the Logistic class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

In [ ]:
"""Logistic regression model."""

import numpy as np


class Logistic:
  def __init__(self, lr: float, epochs: int, threshold: float):
    """Initialize a new classifier.

    Parameters:
        lr: the learning rate
        epochs: the number of epochs to train for
    """
    self.w = None  # TODO: change this
    self.lr = lr
    self.epochs = epochs
    self.threshold = threshold

  def sigmoid(self, z: np.ndarray) -> np.ndarray:
    """Sigmoid function.

    Parameters:
        z: the input

    Returns:
        the sigmoid of the input
    """
    # TODO: implement me
    return 1 / (1 + np.exp(-z))

  def train(self, X_train: np.ndarray, y_train: np.ndarray):
    """Train the classifier.

    Use the logistic regression update rule as introduced in lecture.

    Parameters:
        X_train: a numpy array of shape (N, D) containing training data;
            N examples with D dimensions
        y_train: a numpy array of shape (N,) containing training labels
    """
    # TODO: implement me
    N, D = X_train.shape
    self.w = np.random.randn(D)

    for epoch in range(self.epochs):
      z = np.dot(X_train, self.w)
      y_hat = self.sigmoid(z)
      grad = (1 / N) * np.dot(X_train.T, y_hat - y_train)
      self.w -= self.lr * grad
      self.lr *= np.exp(-1 * (epoch * 0.002))

  def predict(self, X_test: np.ndarray) -> np.ndarray:
    """Use the trained weights to predict labels for test data points.

    Parameters:
        X_test: a numpy array of shape (N, D) containing testing data;
            N examples with D dimensions

    Returns:
        predicted labels for the data in X_test; a 1-dimensional array of
            length N, where each element is an integer giving the predicted
            class.
    """
    # TODO: implement me
    z = np.dot(X_test, self.w)
    y_hat = self.sigmoid(z)
    pred = [1 if i > self.threshold else 0 for i in y_hat]
    return pred

### Training Logistic Classifer

In [ ]:
learning_rate = 0.00001
n_epochs = 180
threshold = 0.3

# 0.0006, 0.0001, 180, 0.5 - 85%
# 0.002, 0.00001, 180, 0.5 - 92%
# 0.002, 0.00001, 180, 0.3 - 98%

lr = Logistic(learning_rate, n_epochs, threshold)
lr.train(X_train_RICE, y_train_RICE)

<ipython-input-269-03af0186b590>:29: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


In [ ]:
pred_lr = lr.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_lr, y_train_RICE)))

The training accuracy is given by: 98.671066


### Validate Logistic Classifer

In [ ]:
pred_lr = lr.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_RICE)))

The validation accuracy is given by: 98.680231


### Test Logistic Classifier

In [ ]:
pred_lr = lr.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_RICE)))

The testing accuracy is given by: 98.542755
